# Logistic Regression Model: Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with a model created from scratch.

In this notebook we will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.
## Fire up [Sframe](https://github.com/dato-code/SFrame)

In [ ]:
import sframe

## Loading data
amazon_reviews.csv es la unión de varios datasets (ver notebook Proyecto_DSC_DIRJAA)

In [ ]:
reviews = sframe.SFrame('amazon_reviews.csv')

# Exploring Data 1/6
Let us quickly explore more of this dataset.
1. We count the number of positive and negative reviews 
2. list the first 10 products in the dataset.
3. **TODO** Histogram

In [ ]:
len(reviews)

In [ ]:
reviews.print_rows(num_rows=2, num_columns=10)

# Data Engineering: defining reviews with positive or negative sentiment

We'll call data engineering, just defining what is a positive and negative sentiment. So let's do that right now. So in the subsection we're gonna define what's a positive and a negative sentiment.
And so I'm gonna make an arbitrary choice here:
1. Let's say that things that 4, 5 stars are things that people liked. So those are positives. 
2. Things that 1 and 2 stars are negative. 
3. ignore all 3 star reviews.
So I'm gonna say a positive sentiment equals 4 star or 5 star reviews. So let's go ahead and add a new column to our table that defines the actual sentiment. So products new column called sentiment.

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [ ]:
reviews = reviews[reviews['overall'] != 3]
len(reviews)

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [ ]:
reviews['sentiment'] = reviews['overall'].apply(lambda rating : +1 if rating > 3 else -1)
reviews.print_rows(num_rows=2, num_columns=10)

### Subsample dataset to make sure classes are balanced
Just as we did in the previous assignment, we will undersample the larger class (safe loans) in order to balance out our dataset. This means we are throwing away many data points. We use `seed=1` so everyone gets the same results.

In [ ]:
positive_reviews_raw = reviews[reviews['sentiment'] == 1]
negative_reviews_raw  = reviews[reviews['sentiment'] == -1]

# Undersample the reviews.
percentage = len(negative_reviews_raw)/float(len(positive_reviews_raw))
negative_reviews = negative_reviews_raw
positive_reviews = positive_reviews_raw.sample(percentage, seed=1)
products = negative_reviews_raw.append(positive_reviews)

print "Percentage of positive reviews             :", len(positive_reviews) / float(len(products))
print "Percentage of negative reviews             :", len(negative_reviews) / float(len(products))
print "Total number of reviews in our new dataset :", len(products)

# Exploring Data 2/6
Let us quickly explore more of this dataset.
3. We count the number of positive and negative reviews.

**TODO**: Modify the subset to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

In [ ]:
print '# of positive reviews =', len(reviews[reviews['sentiment']==1])
print '# of negative reviews =', len(reviews[reviews['sentiment']==-1])

In [ ]:
print '# of positive balanced reviews =', len(products[products['sentiment']==1])
print '# of negative balanced reviews =', len(products[products['sentiment']==-1])

# Reviews Extraction Phase: data preparation. 
## (desarrollado en el notebook Proyecto_DSC_DIRJAA)
**Note:** 
 - column review_clean with text cleaning developed in Proyecto_DSC_DIRJAA notebook
 - building features: la lista de palabras important_words.json se confecciona en el nb Proyecto_DSC_DIRJAA

**Note:** There are several ways of doing this. We use the built-in *count* function for Python lists. Each **review without punctuation, stopwords, etc** string is first split into individual words and the number of occurances of a given word is counted.
1. Transform the reviews into word-counts (only for **important_words**, without punctuation, stopwords, etc)
2. For each word in **important_words**, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in **important_words** which keeps a count of the number of times the respective word occurs in the review text.

Now, we will load these words from this JSON file:

In [ ]:
import json
with open('important_words.json', 'r') as f: # Reads the list of words
    important_words = json.load(f)
important_words = [str(s) for s in important_words]

In [ ]:
print important_words

In [ ]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

# Exploring Data 3/6

The SFrame **products** now contains one column for each of the **important_words**. As an example, the column **perfect** contains a count of the number of times the word **perfect** occurs in each of the reviews.

In [ ]:
products['perfect']

Now, write some code to compute the number of product reviews that contain the word **perfect**.
* First create a column called `contains_perfect` which is set to 1 if the count of the word **perfect** (stored in column **perfect**) is >= 1.
* Sum the number of 1s in the column `contains_perfect`.

In [ ]:
products['contains_perfect'] = products['perfect'].apply(lambda s : +1 if s >= 1 else 0)

In [ ]:
products['contains_perfect'].sum()

# Implementing logistic regression from scratch

## link function (estimating conditional probability)

Recall from lecture that the link function is given by:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))},
$$

where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review  $\mathbf{x}_i$.

In [ ]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''

def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    scores = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1.0 / (1.0 + np.exp(-scores))

    return predictions

How the link function works with matrix algebra?

Since the word counts are stored as columns in **feature_matrix**, each $i$-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\
h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N)
\end{array}
\right]
$$

By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^T h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$.
$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
h(\mathbf{x}_1)^T \\
h(\mathbf{x}_2)^T \\
\vdots \\
h(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
h(\mathbf{x}_1)^T\mathbf{w} \\
h(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
h(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

## Compute derivative of log likelihood with respect to a single coefficient

Recall:
$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

Function that computes the derivative of log likelihood with respect to a single coefficient $w_j$. The function accepts two arguments:
* `errors` vector containing $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ for all $i$.
* `feature` vector containing $h_j(\mathbf{x}_i)$  for all $i$. 

In [ ]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
    
    # Return the derivative
    return derivative

We introduced a transformation of this likelihood---called the log likelihood---that simplifies the derivation of the gradient and is more numerically stable.  Due to its numerical stability, we will use the log likelihood instead of the likelihood to assess the algorithm.

The log likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

$$\ell\ell(\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1)\mathbf{w}^T h(\mathbf{x}_i) - \ln\left(1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))\right) \Big) $$

Function to compute the log likelihood for the entire dataset.

In [ ]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    logexp = np.log(1. + np.exp(-scores))
    
    # Simple check to prevent overflow
    mask = np.isinf(logexp)
    logexp[mask] = -scores[mask]
    
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

## Taking gradient steps
Now we are ready to implement our own logistic regression. 

Function to solve the logistic regression model using gradient ascent:

In [ ]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in xrange(max_iter):

        # Predict P(y_i = +1|x_i,w) using your predict_probability() function
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)
        
        # Compute the errors as indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop over each coefficient
            
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j].
            # Compute the derivative for coefficients[j]. Save it in a variable called derivative
            derivative = np.dot(errors,feature_matrix[:,j])
            
            # add the step size times the derivative to the current coefficient
            coefficients[j] = coefficients[j] + derivative*step_size
        
        # Checking whether log likelihood is increasing
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print 'iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp)
    return coefficients

# Resolving a sentiment classifier with logistic regression

## Split data into training and test sets
Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [ ]:
train_data, test_data = products.random_split(.8, seed=1)

In [ ]:
# Warning: This may take a few minutes...
print '# of total reviews =', len(products)
print '# of positive reviews on all data =', len(products[products['sentiment']==1])
print '# of negative reviews on all data =', len(products[products['sentiment']==-1])

In [ ]:
# Warning: This may take a few minutes...
print '# of train_data reviews =', len(train_data)
print '# of positive reviews on train data =', len(train_data[train_data['sentiment']==1])
print '# of negative reviews on train data =', len(train_data[train_data['sentiment']==-1])

In [ ]:
# Warning: This may take a few minutes...
print '# of test_data reviews =', len(test_data)
print '# of positive reviews on test data =', len(test_data[test_data['sentiment']==1])
print '# of negative reviews on test data =', len(test_data[test_data['sentiment']==-1])

## SFrame to NumPy array
NumPy is a powerful library for doing matrix manipulation. Let us convert our data to matrices and then implement our algorithms with matrices.

Function that extracts columns from an SFrame and converts them into a NumPy array. Two arrays are returned: one representing features and another representing class labels. The feature matrix includes an additional column 'intercept' to take account of the intercept term.

In [ ]:
import numpy as np
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

def get_numpy_feature_matrix(data_sframe, features):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    return(feature_matrix)

# Training Set

Let us convert the train_data into NumPy arrays.

In [ ]:
# Warning: This may take a few minutes...
train_feature_matrix, train_sentiment = get_numpy_data(train_data, important_words, 'sentiment') 

In [ ]:
train_feature_matrix.shape

## Creating the sentiment classifier on the training data

In [ ]:
# Warning: This may take a few minutes...
#Atención el número de ceros debe coincidir con el número de palabras más uno.
sentiment_model_coefficients = logistic_regression(train_feature_matrix, train_sentiment, initial_coefficients=np.zeros(train_feature_matrix.shape[1]),
                                   step_size=1e-7, max_iter=301)

## Class predictions from scores

Class predictions for a data point $\mathbf{x}$ can be computed from the coefficients $\mathbf{w}$ using the following formula:
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Now, we will write some code to compute class predictions. We will do this in two steps:
* **Step 1**: First compute the **scores** using **feature_matrix** and **coefficients** using a dot product.
* **Step 2**: Using the formula above, compute the class predictions from the scores.

Step 1 can be implemented as follows:

In [ ]:
# Step 1: Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(train_feature_matrix, sentiment_model_coefficients)

In [ ]:
# Step 2: compute the class predictions using the **scores** obtained above:
train_sentiment_predictions = map((lambda score: +1 if score > 0 else -1), scores)

## Measuring accuracy of the model

We will now measure the classification accuracy of the model. 
In accuracy, instead of measuring the number of errors, we measure the number of correct classifications.
So the ratio here is number of correct divided by total number of sentences. 
In terms of accuracy, the best possible value is 1, I've got all the sentences right. 
The classification accuracy can be computed as follows:

$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$

In [ ]:
num_mistakes = (train_sentiment != train_sentiment_predictions).sum()
accuracy = 1.0 * (len(train_data) - num_mistakes) / len(train_data)
print "-----------------------------------------------------"
print '# Reviews   correctly classified =', len(train_data) - num_mistakes
print '# Reviews incorrectly classified =', num_mistakes
print '# Reviews total                  =', len(train_data)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

# Exploring Data 4/6
## Which words contribute most to positive & negative sentiments?

We were able to compute the "**most positive words**". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:
* Treat each coefficient as a tuple, i.e. (**word**, **coefficient_value**).
* Sort all the (**word**, **coefficient_value**) tuples by **coefficient_value** in descending order.

In [ ]:
sentiment_model_coefficients_without_intercept = list(sentiment_model_coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, sentiment_model_coefficients_without_intercept)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

Now, **word_coefficient_tuples** contains a sorted list of (**word**, **coefficient_value**) tuples. The first 10 elements in this list correspond to the words that are most positive.

### Ten "most positive" words

Now, we compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment.

In [ ]:
word_coefficient_tuples[0:10]

### Ten "most negative" words

Next, we repeat this exercise on the 10 most negative words.  That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

In [ ]:
word_coefficient_tuples[len(word_coefficient_tuples)-10:len(word_coefficient_tuples)]

# Test Set. Making predictions with logistic regression
Now that a model is trained, we can make predictions on the **test data**.

In [ ]:
# We need to convert test_data into the sparse matrix format first.
# Warning: This may take a few minutes...
test_feature_matrix, test_sentiment = get_numpy_data(test_data, important_words, 'sentiment')

In [ ]:
# Step 1: Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(test_feature_matrix, sentiment_model_coefficients)

In [ ]:
# Step 2: compute the class predictions using the **scores** obtained above:
test_sentiment_predictions = map((lambda score: +1 if score > 0 else -1), scores)

## Accuracy on Test Set

In [ ]:
num_test_mistakes = (test_sentiment != test_sentiment_predictions).sum()
accuracy = 1.0 * (len(test_data) - num_test_mistakes) / len(test_data)
print "-----------------------------------------------------"
print '# Reviews   correctly classified =', len(test_data) - num_test_mistakes
print '# Reviews incorrectly classified =', num_test_mistakes
print '# Reviews total                  =', len(test_data)
print "-----------------------------------------------------"
print 'Accuracy = %.2f' % accuracy

# Exploring Data 5/6
## Applying the learned model to understand sentiment for reviews

In [ ]:
test_data['predicted_sentiment'] = scores

### Most positive reviews

In [ ]:
test_data = test_data.sort('predicted_sentiment', ascending=False)

In [ ]:
test_data[['predicted_sentiment','sentiment','overall','reviewText']][0:5]

In [ ]:
# Most positive review
test_data[0]['reviewText']

### Most negative reviews

In [ ]:
test_data = test_data.sort('predicted_sentiment', ascending=True)

In [ ]:
test_data[['predicted_sentiment','sentiment','overall','reviewText']][0:5]

In [ ]:
# Most negative review
test_data[0]['reviewText']

# Exploring Data 6/6: 
# TODO: Applying the learned model to discover insights on twitter

## Tweets Extraction Phase: data preparation. 
### (desarrollado en el notebook Proyecto_DSC_DIRJAA)
**Note:** 
 - column text_clean with text cleaning developed in Proyecto_DSC_DIRJAA notebook

In [ ]:
tweets_data = sframe.SFrame('us_tweets.csv')

In [ ]:
for word in important_words:
    tweets_data[word] = tweets_data['text_clean'].apply(lambda s : s.split().count(word))

In [ ]:
# We need to convert test_data into the sparse matrix format first.
# Warning: This may take a few minutes...
tweets_feature_matrix  = get_numpy_feature_matrix(tweets_data, important_words) 

In [ ]:
# Step 1: Compute the scores as a dot product between feature_matrix and coefficients.
scores = np.dot(tweets_feature_matrix, sentiment_model_coefficients)

In [ ]:
tweets_data['predicted_sentiment'] = scores

## Export to json array to d3 visualization

## Five most positive tweets

In [ ]:
tweets_data = tweets_data.sort('predicted_sentiment', ascending=False)

In [ ]:
tweets_data[['predicted_sentiment','text']][0:5]

In [ ]:
positive_tweets_data = tweets_data[0:5]

In [ ]:
positive_tweets_data.export_json('d3/data/USA-positive.json')

## Five most negative tweets

In [ ]:
tweets_data = tweets_data.sort('predicted_sentiment', ascending=True)

In [ ]:
tweets_data[['predicted_sentiment','text']][0:5]

In [ ]:
negative_tweets_data = tweets_data[0:5]

In [ ]:
negative_tweets_data.export_json('d3/data/USA-negative.json')

# Most positives and negatives tweets of EEUU dashboard

In [1]:
from IPython.display import IFrame

def serve_html():
    fn= './d3/html/d3_6_US_map_tooltips.html'
    return IFrame(fn, 985, 570)

In [2]:
serve_html()